# 🏠 House Price Prediction - Hybrid Model Training

## Overview
This notebook demonstrates the **Hybrid CNN + XGBoost Pipeline** for house price prediction:
1. **PyTorch Model**: ResNet18 CNN + Tabular MLP fusion (256 + 64 = 320 features)
2. **XGBoost Model**: Trained on deep features extracted from PyTorch model
3. **Automatic Selection**: Best model (by R²) generates final predictions

## Pipeline Steps
1. Load data and satellite images
2. Train PyTorch HybridMultimodalModel
3. Extract 320-dim deep features
4. Train XGBoost on those features
5. Compare models and pick winner
6. Generate Grad-CAM visualizations
7. Save `submission.csv` using best model

In [ ]:
# Import libraries
import sys
sys.path.append('..')

import os
import math
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import xgboost as xgb
import matplotlib.pyplot as plt
from tqdm import tqdm

from src.config import cfg
from src.data_fetcher import download
from src.datasets import HouseDataset
from src.model import HybridMultimodalModel
from src.gradcam import GradCAM

# Set seed for reproducibility
torch.manual_seed(cfg.seed)
np.random.seed(cfg.seed)

print(f"Using device: {cfg.device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

ModuleNotFoundError: No module named 'torch'

## 1. Load Data

In [ ]:
# Load train and test data
train_df = pd.read_excel(f"../{cfg.train_xlsx}")
test_df = pd.read_excel(f"../{cfg.test_xlsx}")

print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")
print(f"\nTabular features: {cfg.tab_feats}")

# Fetch/load satellite images (uses free ESRI - no API key!)
print("\n📡 Fetching satellite images...")
img_paths = download(pd.concat([train_df, test_df], axis=0))

# Filter rows with valid images
train_df = train_df[train_df["id"].isin(img_paths.keys())]
test_df = test_df[test_df["id"].isin(img_paths.keys())]
print(f"\nTrain samples with images: {len(train_df)}")
print(f"Test samples with images: {len(test_df)}")

## 2. Prepare Data & Create DataLoaders

In [ ]:
# Scale tabular features
scaler = StandardScaler()
scaler.fit(train_df[cfg.tab_feats].astype(float))

# Train-validation split
tr_df, val_df = train_test_split(train_df, test_size=cfg.val_split, random_state=cfg.seed)
print(f"Training set: {len(tr_df)}")
print(f"Validation set: {len(val_df)}")

# Create datasets
tr_ds = HouseDataset(tr_df, img_paths, scaler, train=True)
val_ds = HouseDataset(val_df, img_paths, scaler, train=True)
te_ds = HouseDataset(test_df, img_paths, scaler, train=False)

# Create dataloaders
tr_loader = DataLoader(tr_ds, batch_size=cfg.batch_size, shuffle=True, num_workers=cfg.num_workers)
val_loader = DataLoader(val_ds, batch_size=cfg.batch_size, shuffle=False, num_workers=cfg.num_workers)
te_loader = DataLoader(te_ds, batch_size=cfg.batch_size, shuffle=False, num_workers=cfg.num_workers)

print(f"\nDataLoaders created with batch_size={cfg.batch_size}")

## 3. Train PyTorch HybridMultimodalModel

The model architecture:
- **Visual Hemisphere**: ResNet18 → 256 features (with BatchNorm + Dropout)
- **Logical Hemisphere**: Tabular MLP → 64 features (with BatchNorm + Dropout)  
- **Combined**: 320 features → Regressor head → Price prediction

In [ ]:
# Helper functions
def rmse(pred, true):
    return math.sqrt(mean_squared_error(true, pred))

def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for img, tab, y in loader:
        img, tab, y = img.to(device), tab.to(device), y.to(device)
        optimizer.zero_grad()
        pred = model(img, tab)
        loss = criterion(pred, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * len(y)
    return total_loss / len(loader.dataset)

@torch.no_grad()
def eval_model(model, loader, criterion, device):
    model.eval()
    ys, ps = [], []
    total_loss = 0
    for img, tab, y in loader:
        img, tab, y = img.to(device), tab.to(device), y.to(device)
        pred = model(img, tab)
        loss = criterion(pred, y)
        total_loss += loss.item() * len(y)
        ys.append(y.cpu().numpy())
        ps.append(pred.cpu().numpy())
    ys = np.concatenate(ys)
    ps = np.concatenate(ps)
    return total_loss / len(loader.dataset), rmse(ps, ys), r2_score(ys, ps)

# Initialize model
device = cfg.device
model = HybridMultimodalModel(tabular_input_dim=len(cfg.tab_feats)).to(device)
optimizer = optim.Adam(model.parameters(), lr=cfg.lr, weight_decay=cfg.weight_decay)
criterion = nn.MSELoss()

print(f"Model initialized on {device}")
print(f"Feature dimension: 256 (visual) + 64 (tabular) = 320 combined")

## 4. Training Loop

Train for configured epochs and track best model by RMSE:

In [ ]:
# Training loop
best_rmse = float("inf")
best_r2 = float("-inf")
history = {"train_loss": [], "val_rmse": [], "val_r2": []}

print(f"Training for {cfg.epochs} epochs...")
print("-" * 60)

for epoch in tqdm(range(cfg.epochs), desc="Training"):
    tr_loss = train_one_epoch(model, tr_loader, optimizer, criterion, device)
    val_loss, val_rmse, val_r2 = eval_model(model, val_loader, criterion, device)
    
    history["train_loss"].append(tr_loss)
    history["val_rmse"].append(val_rmse)
    history["val_r2"].append(val_r2)
    
    print(f"Epoch {epoch+1}/{cfg.epochs} | Train Loss: {tr_loss:.4f} | Val RMSE: ${val_rmse:,.0f} | Val R²: {val_r2:.4f}")
    
    if val_rmse < best_rmse:
        best_rmse = val_rmse
        best_r2 = val_r2
        torch.save({"model": model.state_dict(), "epoch": epoch+1, "best_rmse": best_rmse, "best_r2": best_r2}, 
                   f"../models/best_model.pt")
        print(f"   ✅ New best model saved!")

print("-" * 60)
print(f"✅ Best PyTorch Model: RMSE=${best_rmse:,.0f}, R²={best_r2:.4f}")

## 5. Train XGBoost on Deep Features

Extract 320-dim features from PyTorch model and train XGBoost:

In [ ]:
# Extract features for XGBoost
def extract_features(model, loader, device, include_targets=True):
    model.eval()
    features_list, targets_list, ids_list = [], [], []
    with torch.no_grad():
        for batch in loader:
            if include_targets:
                img, tab, y = batch
                targets_list.extend(y.numpy())
            else:
                img, tab, pid = batch
                ids_list.extend(pid.tolist() if hasattr(pid, 'tolist') else list(pid))
            img, tab = img.to(device), tab.to(device)
            feats = model.get_features_only(img, tab)
            features_list.append(feats)
    features = np.vstack(features_list)
    if include_targets:
        return features, np.array(targets_list)
    return features, ids_list

# Extract features
X_train, y_train = extract_features(model, tr_loader, device)
X_val, y_val = extract_features(model, val_loader, device)
print(f"Training features: {X_train.shape}")
print(f"Validation features: {X_val.shape}")

# Train XGBoost
xgb_model = xgb.XGBRegressor(
    n_estimators=200, learning_rate=0.1, max_depth=6,
    min_child_weight=1, subsample=0.8, colsample_bytree=0.8,
    random_state=cfg.seed, n_jobs=-1
)
xgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
print("✅ XGBoost Training Complete")

## 6. Model Comparison & Winner Selection

Compare PyTorch vs XGBoost and select best by R² score:

In [ ]:
# PyTorch predictions on validation
model.eval()
y_pred_pt = []
y_true = []
with torch.no_grad():
    for img, tab, y in val_loader:
        img, tab = img.to(device), tab.to(device)
        pred = model(img, tab)
        y_pred_pt.extend(pred.cpu().numpy())
        y_true.extend(y.numpy())
y_pred_pt = np.array(y_pred_pt)
y_true = np.array(y_true)

# PyTorch metrics
pt_rmse = rmse(y_pred_pt, y_true)
pt_r2 = r2_score(y_true, y_pred_pt)
pt_mae = mean_absolute_error(y_true, y_pred_pt)

# XGBoost metrics
y_pred_xgb = xgb_model.predict(X_val)
xgb_rmse_val = rmse(y_pred_xgb, y_val)
xgb_r2 = r2_score(y_val, y_pred_xgb)
xgb_mae = mean_absolute_error(y_val, y_pred_xgb)

# Print comparison
print("=" * 60)
print("🏆 MODEL COMPARISON (Validation Set)")
print("=" * 60)
print(f"\n{'Model':<30} {'RMSE':>12} {'R²':>10} {'MAE':>12}")
print("-" * 60)
print(f"{'PyTorch (CNN + Tabular)':<30} ${pt_rmse:>10,.0f} {pt_r2:>10.4f} ${pt_mae:>10,.0f}")
print(f"{'XGBoost (on Deep Features)':<30} ${xgb_rmse_val:>10,.0f} {xgb_r2:>10.4f} ${xgb_mae:>10,.0f}")
print("-" * 60)

# Declare winner
if xgb_r2 > pt_r2:
    print(f"\n🏆 WINNER: XGBoost on Deep Features! (R²: {xgb_r2:.4f})")
    winner = "xgboost"
else:
    print(f"\n🏆 WINNER: PyTorch End-to-End Model! (R²: {pt_r2:.4f})")
    winner = "pytorch"

## 7. Generate Test Predictions

Save `submission.csv` using the best model:

In [ ]:
# Extract test features
X_test, test_ids = extract_features(model, te_loader, device, include_targets=False)

# Generate predictions from the model with best R² score
if xgb_r2 > pt_r2:
    best_preds = xgb_model.predict(X_test)
    best_model_name = "XGBoost"
    best_r2_score = xgb_r2
else:
    best_preds = []
    model.eval()
    with torch.no_grad():
        for img, tab, pid in te_loader:
            img, tab = img.to(device), tab.to(device)
            pred = model(img, tab).cpu().numpy()
            best_preds.extend(pred.tolist())
    best_model_name = "PyTorch"
    best_r2_score = pt_r2

# Save only the best model's submission
os.makedirs("../outputs", exist_ok=True)
submission = pd.DataFrame({"id": test_ids, "predicted_price": best_preds})
submission.to_csv("../outputs/submission.csv", index=False)
print(f"✅ Saved: outputs/submission.csv (using {best_model_name} with R²: {best_r2_score:.4f})")
print(f"\nFirst 5 predictions:")
print(submission.head())

## 8. Grad-CAM Visualization

Generate heatmaps showing which parts of satellite images influence predictions:

In [ ]:
import cv2
from src.train import run_gradcam

# Generate Grad-CAM visualizations
run_gradcam(model, val_ds, "../outputs")
print("✅ Grad-CAM visualizations saved to outputs/gradcam/")

# Display a sample
gradcam_dir = "../outputs/gradcam"
if os.path.exists(gradcam_dir):
    samples = [f for f in os.listdir(gradcam_dir) if f.endswith('.png')][:4]
    if samples:
        fig, axes = plt.subplots(1, len(samples), figsize=(16, 4))
        for ax, sample in zip(axes, samples):
            img = plt.imread(os.path.join(gradcam_dir, sample))
            ax.imshow(img)
            ax.set_title(sample.replace('.png', ''))
            ax.axis('off')
        plt.suptitle("Grad-CAM: What the model sees", fontsize=14)
        plt.tight_layout()
        plt.show()

## 9. Training History Visualization

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].plot(history["train_loss"], 'b-', label='Train Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(history["val_rmse"], 'r-', label='Val RMSE')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('RMSE ($)')
axes[1].set_title('Validation RMSE')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

axes[2].plot(history["val_r2"], 'g-', label='Val R²')
axes[2].set_xlabel('Epoch')
axes[2].set_ylabel('R²')
axes[2].set_title('Validation R²')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "=" * 60)
print("✅ TRAINING COMPLETE!")
print(f"   Winner: {winner.upper()} (R²: {xgb_r2:.4f if winner == 'xgboost' else pt_r2:.4f})")
print("=" * 60)